Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [36]:
from itertools import combinations
from collections import namedtuple, defaultdict
from random import choice
from copy import deepcopy
import time
from typing import Callable

from tqdm.auto import tqdm
import numpy as np

In [2]:
State = namedtuple('State', ['x', 'o'])

In [3]:
MAGIC = [2, 7, 6, 9, 5, 1, 4, 3, 8]

In [4]:
def print_board(pos):
    """Nicely prints the board"""
    for r in range(3):
        for c in range(3):
            i = r * 3 + c
            if MAGIC[i] in pos.x:
                print('X', end='')
            elif MAGIC[i] in pos.o:
                print('O', end='')
            else:
                print('.', end='')
        print()
    print()

In [5]:
def win(elements):
    """Checks is elements is winning"""
    return any(sum(c) == 15 for c in combinations(elements, 3))

def state_value(pos: State):
    """Evaluate state: +1 first player wins"""
    if win(pos.x):
        return 1
    elif win(pos.o):
        return -1
    else:
        return 0
    
    

In [6]:
def random_game():
    trajectory = list()
    state = State(set(), set())
    available = set(range(1, 9+1))
    while available:
        x = choice(list(available))
        state.x.add(x)
        trajectory.append(deepcopy(state))
        available.remove(x)
        if win(state.x) or not available:
            break

        o = choice(list(available))
        state.o.add(o)
        trajectory.append(deepcopy(state))
        available.remove(o)
        if win(state.o):
            break
    return trajectory

In [26]:
random_game()

[State(x={6}, o=set()),
 State(x={6}, o={1}),
 State(x={9, 6}, o={1}),
 State(x={9, 6}, o={1, 5}),
 State(x={8, 9, 6}, o={1, 5}),
 State(x={8, 9, 6}, o={1, 5, 7}),
 State(x={8, 9, 4, 6}, o={1, 5, 7}),
 State(x={8, 9, 4, 6}, o={1, 3, 5, 7})]

In [18]:
value_dictionary = defaultdict(float)
hit_state = defaultdict(int)
epsilon = 0.001

for steps in tqdm(range(500_000)):
    trajectory = random_game()
    final_reward = state_value(trajectory[-1])
    for state in trajectory:
        hashable_state = (frozenset(state.x), frozenset(state.o))
        hit_state[hashable_state] += 1
        value_dictionary[hashable_state] = value_dictionary[
            hashable_state
        ] + epsilon * (final_reward - value_dictionary[hashable_state])
        


  0%|          | 0/500000 [00:00<?, ?it/s]

In [35]:
sorted(value_dictionary.items(), key=lambda e: e[1], reverse=True)[50:60]

[((frozenset({5}), frozenset({3})), 0.5880254689089696),
 ((frozenset({5}), frozenset({7})), 0.5786611266629809),
 ((frozenset({1, 3, 4, 5}), frozenset({2, 7, 8, 9})), 0.5765965561633927),
 ((frozenset({1, 2, 7, 8}), frozenset({3, 4, 6, 9})), 0.5753238011708255),
 ((frozenset({5, 6, 8, 9}), frozenset({1, 2, 3, 7})), 0.573620841105109),
 ((frozenset({1, 2, 4, 5}), frozenset({3, 6, 7, 9})), 0.5727668019421914),
 ((frozenset({1, 4, 6, 7}), frozenset({3, 5, 8, 9})), 0.5727668019421914),
 ((frozenset({1, 6, 7, 9}), frozenset({2, 3, 4, 5})), 0.5723391410832747),
 ((frozenset({2, 3, 5, 6}), frozenset({1, 4, 7, 9})), 0.5723391410832747),
 ((frozenset({1, 2, 4, 6}), frozenset({3, 5, 8, 9})), 0.5710535882583385)]